# Setup

## Imports

In [2]:
using Pkg
Pkg.activate("SCC", shared=true)


import QuantumToolbox as qt
import SuperconductingCavities as SC
import CairoMakie as cm
using Revise
using Dates
import JSON3

using ProgressMeter

import Optim as opt

using Logging
using MiniLoggers

  Activating project at `~/.julia/environments/SCC`
[ Info: new driver key :netcdf, updating backendlist.


## Logger Setups

In [3]:
# MiniLogger(minlevel = MiniLoggers.Info) |> global_logger
InfoLogger = MiniLogger(minlevel = MiniLoggers.Info)
ProgressLogger = MiniLogger(minlevel = LogLevel(-1))
DebugLogger = MiniLogger(minlevel = MiniLoggers.Debug)

global_logger(ProgressLogger)

ConsoleLogger(IJulia.IJuliaStdio{Base.PipeEndpoint}(IOContext(Base.PipeEndpoint(RawFD(40) open, 0 bytes waiting))), Info, Base.CoreLogging.default_metafmt, true, 0, Dict{Any, Int64}())

# Loading Model and json

In [48]:
Readout = SC.Circuits.Transmon_Resonators.load("ModelSaves/Readout/Readout.json");
proj_dict = SC.Utils.Get_Projection_Ops(Readout.dressed_states);

device_config = JSON3.read("device_config_files/config_multimode.json", Dict{Any, Any});



In [49]:
GLOBAL_SAVE_CALIBRATE = true;

In [50]:
Readout.params

Dict{Any, Any} with 17 entries:
  "d_t"          => 1
  "Nrs"          => Any[10]
  "E_C"          => 0.102833
  "Cavity_Names" => Any["Resonator"]
  "E_J"          => 26.9698
  "Nt_cut"       => 60
  "E_oscs"       => Any[7.25]
  "Nt"           => 10
  "ModelType"    => "TransmonResonators"
  "ng"           => 0
  "kappa_tc"     => 1.78571e-5
  "kappa_td"     => 1.2348e-5
  "d_r"          => Any[0]
  "kappa_cc"     => Any[1.0e-6]
  "Model_Name"   => "Readout"
  "gs"           => Any[0.0843756]
  "Save_Path"    => "ModelSaves/Readout/"

In [56]:
test = SC.Dynamics.Get_Drive_Coef([1,2,3], 0.1, return_ℂ=true)
test(0.1)

0.508320369231526 - 1.5644492319947136im

# Fitting g

In [44]:
χ_e(model) = (model.dressed_energies[(1,1)]-model.dressed_energies[(0,1)])-(model.dressed_energies[(1,0)]-model.dressed_energies[(0,0)]);
χ_e_target = -0.8796349178368246./1000

initial_g = 0.02

E_C = Readout.params["E_C"]
E_J = Readout.params["E_J"]
E_osc = Readout.params["E_oscs"]
function function_to_minimize_maker(E_osc, χ_e_target)
    function to_min(g)
        model = SC.Circuits.Transmon_Resonators.init(E_C, E_J, E_osc, g, 10, 10)
        return abs(χ_e_target - χ_e(model))
    end
    return to_min
end

function_to_minimize_maker (generic function with 2 methods)

In [45]:
to_min = function_to_minimize_maker(E_osc, χ_e_target)
min_res = opt.optimize(to_min, [0.01])#, opt.LBFGS())

 * Status: success

 * Candidate solution
    Final objective value:     1.362561e-08

 * Found with
    Algorithm:     Nelder-Mead

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    9
    f(x) calls:    21


In [46]:
println(min_res.minimum)
println(min_res.minimizer)

1.362560908546584e-8
[0.0843756103515625]


In [47]:
Readout.params["gs"] = min_res.minimizer
SC.Utils.save_model(Readout)

# Calibrating Pulses